In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import matplotlib.image as mpimg
from numpy import linalg
import json

In [2]:
class VisualOdometry():
    
    def read_ground_truth_file(self,path):
        with open(path, "r") as f:
            ground_truth = json.load(f)
        return ground_truth
    
    def __init__(self):
        path = './kitti_dataset/imgs/*'
        self.images_list = glob.glob(path)
        
        self.ground_truth = self.read_ground_truth_file("ground_truth.json")
        
        self.focal = 718.856
        self.pp = (607.1928, 185.2157)
        
        self.R = np.eye(3)
        self.t = np.zeros((3, 1))
        self.detection_threshold = 2000
        
        #self.detector = cv2.ORB_create(5000)
        
        self.lk_params = {"winSize": (21, 21),
                          "maxLevel": 5}
        
        self.prev_points = np.empty(0)
        self.good_points = 0
        self.idx = 0
        
        self.prev_img = self.read_img()
        
        
    def run(self):
        while self.idx < len(self.images_list)-1 :
        #self.idx apo 0 mexri 4539:
        
            current = self.read_img()
        
            if self.good_points < self.detection_threshold:
                corners = self.detect()
            
            prev_points, good_points = self.track(current,corners)
            
            E = self.find_essential(prev_points, good_points)
            
            R, t = self.find_rt(E, prev_points, good_points)
            
            scale = self.find_scale()
            
            self.t = self.t + scale * self.R.dot(t)
            self.R = self.R.dot(R)
            
            self.prev_points = good_points
            self.good_points = good_points.shape[0]
            
            self.prev_img = current
            
        
    def read_img(self):
        image = mpimg.imread(self.images_list[self.idx])
        image = np.array(image * 255, dtype=np.uint8)
        self.idx += 1
        return image
    
    def detect(self):
        corners = cv2.goodFeaturesToTrack(self.prev_img, mask=None, maxCorners = 100, qualityLevel = 0.03, minDistance = 7, blockSize = 7)
        corners = np.squeeze(corners)
        return corners
    
    def track(self,image,corners):
        next_corners, status, _ = cv2.calcOpticalFlowPyrLK(self.prev_img, image, corners, None, **self.lk_params)
        if next_corners is not None:
            prev_good = corners[status.ravel() == 1]
            good = next_corners[status.ravel() == 1]

        return prev_good,good
    
    def find_essential(self, prev_good, good):
        E, _ = cv2.findEssentialMat(prev_good, good,
                                    focal=self.focal, pp=self.pp,
                                    method = cv2.RANSAC, prob=0.999, threshold=1.0)
        return E
    
    def find_rt(self, E, prev_good, good):
        _, R, t, _ = cv2.recoverPose(E, prev_good, good, focal=self.focal, pp=self.pp)
        return R, t
    
    def find_scale(self):
        prev_position = self.ground_truth[self.idx -1]
        prev_position = np.array(prev_position)
        current_position = self.ground_truth[self.idx]
        current_position = np.array(current_position)
        self.idx += 1
        
        scale = np.linalg.norm(current_position - prev_position)
        return scale
    
    
    

In [5]:
visual_odometry = VisualOdometry()
visual_odometry.run()


In [3]:
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Ορίστε ένα όνομα και ένα μονοπάτι αρχείου
output = './output/test_mapping.mp4'
data = VisualOdometry()
clip = ImageSequenceClip(data.images_list, fps=20)
new_clip = clip.fl_image(process_image) 
%time new_clip.write_videofile(output, audio=False)

IndexError: tuple index out of range